In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import torch
from IPython.display import Image # For displaying images in colab jupyter cell
from PIL import Image
import io
import pandas as pd

In [ ]:
data = pd.read_csv('star_classification.csv')
data = data.drop(['obj_ID', 'run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'plate', 'MJD','fiber_ID'], axis=1)
data = data.rename(columns={'class' : 'obj'})

In [ ]:
# Segment data based on class, note df.loc preserves index of original dataframe
galaxies = data.loc[data.obj=='GALAXY', ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]
qsos = data.loc[data.obj=='QSO', ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]
stars = data.loc[data.obj=='STAR', ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]

print(galaxies.shape, qsos.shape, stars.shape)

(59445, 8) (18961, 8) (21594, 8)


In [ ]:
input_spectra = stars.copy()

In [ ]:
import requests

def get_color_image(ra, dec):
    url = "http://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg"
    params = {
        "ra": ra,
        "dec": dec,
        "scale": 0.1,
        "height": 64,
        "width": 64,
        "opt": ''
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        return response.content  # Return the binary image data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching image: {e}")
        return None

In [ ]:
def binary_to_rgb(image_binary):
    # Convert binary image data to PIL Image object
    pil_image = Image.open(io.BytesIO(image_binary))

    # Convert PIL Image to numpy array
    rgb_array = np.array(pil_image)

    # If the image is grayscale, convert it to RGB by repeating the grayscale values along the third axis
    if len(rgb_array.shape) == 2:
        rgb_array = np.stack((rgb_array,) * 3, axis=-1)

    return rgb_array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### API Call One Image

In [ ]:
# EXAMPLE
# alpha, delta values pasted from spectra Excel sheet
ra = 135.6891066036
dec = 32.4946318397087

# Run both functions
test_image_binary = get_color_image(ra=ra, dec=dec)
test_image = binary_to_rgb(test_image_binary)

#### API Call N Images

In [ ]:
N = 2000

for i in range(N):

    # Define coordinates based off index
    ra= input_spectra['alpha'][i]
    dec = input_spectra['delta'][i]

    # Define file name for image based off index
    filename = 'image' + str(i)

    # API call to retrieve image with particular coordinates
    spectra_image = get_color_image(ra=ra, dec=dec)

    # Save the image locally
    with open('/content/drive/MyDrive/star_images/'+ filename +'.jpg', 'wb') as f:
        f.write(spectra_image)


#### API Call N Images in Class

In [ ]:
N = 10000
indices = np.array(input_spectra.index)
indices[9999]

45500

In [ ]:
for i in indices[6277:N]:

    ra = data['alpha'][i]
    dec = data['delta'][i]

    spectra_image = get_color_image(ra=ra, dec=dec)

    with open('/content/drive/MyDrive/star_images/image'+ str(i) +'.jpg', 'wb') as f:
        try:

            f.write(spectra_image)

        except TypeError:

            spectra_image = get_color_image(ra=ra, dec=dec)
            with open('/content/drive/MyDrive/star_images/image'+ str(i) +'.jpg', 'wb') as f:
                f.write(spectra_image)